## 0) Import Libraries

In [62]:
import os
import pandas as pd
import seaborn as sns
from ipywidgets import interact, DatePicker, IntSlider
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.forecasting.theta import ThetaModel
import warnings
import numpy as np
warnings.filterwarnings("ignore")

## 1) Import Dataset

In [49]:
df         = pd.read_csv('DailyDelhiClimateTrain.csv')
df_test    = pd.read_csv('DailyDelhiClimateTest.csv')
df['date'] = pd.to_datetime(df['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

## 2) Exploratory Data Analysis

### 2.1) Visualise data trend

In [52]:
@interact
def show_time_series(Dataset = ['Train', 'Test'], Attribute=df.columns.tolist()[1:]):
    if Dataset == 'Train':
        from matplotlib.pyplot import figure
        figure(figsize=(20, 5), dpi=80)
        sns.lineplot('date',Attribute,data=df)
        plt.xticks(rotation=0)
        plt.xlabel('\n\nDate')
        plt.ylabel(Attribute + '\n')
        plt.title(Attribute.upper() + '\n')
        
    elif Dataset == 'Test':
        from matplotlib.pyplot import figure
        figure(figsize=(20, 5), dpi=80)
        sns.lineplot('date',Attribute,data=df_test)
        plt.xticks(rotation=0)
        plt.xlabel('\n\nDate')
        plt.ylabel(Attribute + '\n')
        plt.title(Attribute.upper() + '\n')
        
    

interactive(children=(Dropdown(description='Dataset', options=('Train', 'Test'), value='Train'), Dropdown(desc…

### 2.2) Visualise data distributions

In [57]:
@interact
def show_distribution(Dataset = ['Train', 'Test']):
    if Dataset == 'Train':
        figure(figsize=(20, 5), dpi=80)
        sns.pairplot(df)
        
    if Dataset == 'Test':
        figure(figsize=(20, 5), dpi=80)
        sns.pairplot(df_test)

interactive(children=(Dropdown(description='Dataset', options=('Train', 'Test'), value='Train'), Output()), _d…

## 3) Forecast Attributes using Statistical Methods

In [76]:
@interact
def forecast_AR(Date=DatePicker(value=pd.to_datetime('2013-01-01'), description='Start Date'),
                fh = IntSlider(value=6,min=1,max=168, description='Forecast'),
                Attribute=df.columns.tolist()[1:], 
                Algorithm=['AutoRegression', 'ARIMA', 'Exponential Smoothing', 'Theta']):
    
    def check_date(my_date):
        if my_date is None:
            return False
        start_date = pd.to_datetime('2013-01-01')
        end_date = pd.to_datetime("today")
        my_date = pd.to_datetime(my_date)
        return ((my_date >= start_date) & (my_date <= end_date))
    
    if check_date(Date):
        start_date = pd.to_datetime(Date)

    df_filter1 = df.loc[(df['date'].dt.date >=  Date)]
    df_filter1 = df_filter1[['date', Attribute]]
    df_filter1 = df_filter1.set_index('date')
    y1 = df_filter1.squeeze()
    y1.index = pd.DatetimeIndex(y1.index, freq="D")
    
    df_filter2 = df_test.loc[(df_test['date'].dt.date >=  Date)]
    df_filter2 = df_filter2[['date', Attribute]]
    df_filter2 = df_filter2.set_index('date')
    y2 = df_filter2.squeeze()
    y2.index = pd.DatetimeIndex(y2.index, freq="D")
    
       
    if Algorithm == 'AutoRegression':
        model = AutoReg(y1, lags=1)
   
    elif Algorithm == 'ARIMA':
        model = ARIMA(y1, order=(1, 0, 1), seasonal_order=(1,0,1,7))
        
    elif Algorithm == 'Exponential Smoothing':
        model = ExponentialSmoothing(y1, seasonal_periods=7, seasonal='mul')
        
    elif Algorithm == 'Theta':
        model = ThetaModel(y1, period=7, method='multiplicative')

                           
    model = model.fit()
    y_pred = model.forecast(steps=fh)
    
    
    plt.rcParams["figure.figsize"] = (25,10)
    plt.plot(y1,c='black',alpha=0.3,label='Actual', marker='o')
    plt.plot(y_pred,c='red',label='Predict', marker='o')
    ci = 1.96 * np.std(y_pred)/np.sqrt(len(y_pred.index))
    plt.plot(y2,c='black',alpha=0.3,label='Actual', marker='o')
    plt.fill_between(y_pred.index, (y_pred-ci), (y_pred+ci), color='b', alpha=.1)
    plt.xlabel('Date', fontsize=18)
    plt.ylabel(Attribute + '\n', fontsize=18)
    plt.title('\n' + Attribute.upper() + ' FORECAST\n', fontsize=23)
    plt.xticks(rotation=45)
    plt.legend()
#     plt.axvline(pd.Timestamp('2020-12-21'),color='r',linewidth=4)

interactive(children=(DatePicker(value=Timestamp('2013-01-01 00:00:00'), description='Start Date'), IntSlider(…